<a href="https://colab.research.google.com/github/Vijayakrisnan/Complete-Python-3-Bootcamp/blob/master/McDonalds_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

mcdonalds = pd.read_csv('https://raw.githubusercontent.com/terrytangyuan/MSA/master/data/mcdonalds.csv')
list(mcdonalds.columns.values)


In [ ]:
import numpy as np

np.shape(mcdonalds)


In [ ]:
print(mcdonalds.head(3))


In [ ]:
MD_x = mcdonalds.iloc[:, 0:11].values
MD_x = (MD_x == "Yes").astype(int)
rounded_col_means = np.round(np.mean(MD_x, axis=0), 2)

print(rounded_col_means)

In [1]:
from sklearn.decomposition import PCA
# Create a PCA object and fit the data
pca = PCA()
MD_pca = pca.fit_transform(MD_x)

# Print the summary
print("Proportion of Variance Explained:")
print(pca.explained_variance_ratio_)
print("\nPrincipal Components:")
print(pca.components_)


In [ ]:
# Set the number of digits for printing
digits = 1

# Print the PCA results with specified number of digits
np.set_printoptions(precision=digits)
print(np.round(MD_pca, digits))

In [ ]:
import matplotlib.pyplot as plt
# Plotting the PCA results
plt.scatter(MD_pca[:, 0], MD_pca[:, 1], color='grey')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

# Projecting axes onto the PCA plot
# Obtain the principal components' loadings (eigenvectors)
loadings = pca.components_

# Plotting the projected axes
plt.scatter(MD_pca[:, 0], MD_pca[:, 1], color='grey')
plt.xlabel('PC1')
plt.ylabel('PC2')

# Scaling factor for projecting the axes
scale = np.max(np.abs(MD_pca))

# Plotting the projected axes
for i, (x, y) in enumerate(loadings[:2].T * scale):
    plt.arrow(0, 0, x, y, color='r', alpha=0.5)
    plt.text(x, y, f"Feature {i+1}", color='r')

plt.show()

In [2]:
from sklearn.cluster import KMeans

In [ ]:
# Set the random seed
np.random.seed(1234)

# Define the range of cluster numbers to consider
n_clusters_range = range(2, 9)

# Perform K-means clustering with multiple initializations
best_kmeans = None
best_silhouette_score = -1
n_rep = 10

for n_clusters in n_clusters_range:
    silhouette_scores = []

    for _ in range(n_rep):
        # Create a K-means object
        kmeans = KMeans(n_clusters=n_clusters)

        # Fit the K-means model to the data
        kmeans.fit(MD_x)

        # Obtain the Silhouette score
        silhouette_score = metrics.silhouette_score(MD_x, kmeans.labels_)
        silhouette_scores.append(silhouette_score)

    # Average Silhouette score for the current number of clusters
    avg_silhouette_score = np.mean(silhouette_scores)

    # Update the best K-means model if the current score is higher
    if avg_silhouette_score > best_silhouette_score:
        best_silhouette_score = avg_silhouette_score
        best_kmeans = kmeans

# Relabel the clusters
labels = best_kmeans.labels_
unique_labels = np.unique(labels)
n_clusters = len(unique_labels)
relabel_dict = {unique_labels[i]: i for i in range(n_clusters)}
relabelled_labels = [relabel_dict[label] for label in labels]

# Assign the relabelled labels to the cluster result
MD_kmeans = relabelled_labels

In [ ]:
plt.plot(range(2, 9), MD_kmeans, marker='o')
plt.xlabel("number of segments")
plt.ylabel("Cluster label")
plt.title("K-means Clustering")
plt.show()

In [ ]:
from sklearn.utils import resample
from sklearn.metrics import adjusted_rand_score

In [ ]:
# Set the random seed
np.random.seed(1234)

# Create an empty list to store the bootstrap results
bootstrap_results = []

# Iterate over the number of segments
for n_segments in range(2, 9):
    # Create an empty list to store the adjusted Rand index for each bootstrap iteration
    adjusted_rand_indices = []

    # Perform bootstrap resampling and clustering
    for _ in range(100):
        # Bootstrap resampling
        boot_sample = resample(MD_x, random_state=1234)

        # K-means clustering
        kmeans = KMeans(n_clusters=n_segments, random_state=1234)
        labels = kmeans.fit_predict(boot_sample)

        # Calculate the adjusted Rand index
        adjusted_rand = adjusted_rand_score(MD_kmeans["4"], labels)
        adjusted_rand_indices.append(adjusted_rand)

    # Store the bootstrap results for the current number of segments
    bootstrap_results.append(adjusted_rand_indices)

# Plot the bootstrap results
plt.plot(range(2, 9), np.mean(bootstrap_results, axis=1), marker='o')
plt.xlabel("number of segments")
plt.ylabel("adjusted Rand index")
plt.title("Bootstrapping with Flexclust")
plt.show()

# Plot the histogram for the cluster label "4" in the K-means results
plt.hist(MD_kmeans["4"], bins=np.arange(0, 2), edgecolor='black')
plt.xlabel("Cluster label")
plt.ylabel("Frequency")
plt.title("Histogram of Cluster Label '4'")
plt.xlim(0, 1)
plt.show()

In [ ]:
from tslearn.clustering import sliding_window

# Calculate segment stability using sliding_window
segment_stability = sliding_window(MD_x, MD_kmeans["4"])

# Plot segment stability
plt.plot(segment_stability, marker='o')
plt.ylim(0, 1)
plt.xlabel("segment number")
plt.ylabel("segment stability")
plt.title("Segment Stability with Flexclust")
plt.show()



In [ ]:

from sklearn.mixture import GaussianMixture

np.random.seed(1234)
MD_m28 = None
for k in range(2, 9):
    for n in range(10):
        model = GaussianMixture(n_components=k, covariance_type='full')
        model.fit(MD_x)
        if MD_m28 is None or model.bic(MD_x) < MD_m28.bic(MD_x):
            MD_m28 = model


In [ ]:
# Plot information criteria
plt.plot(MD_m28.ICL, label='ICL')
plt.plot(MD_m28.AIC, label='AIC')
plt.plot(MD_m28.BIC, label='BIC')

# Set labels and legend
plt.xlabel('Number of segments')
plt.ylabel('Value of information criteria')
plt.legend()

# Show the plot
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4).fit(MD.x)
gmm = GaussianMixture(n_components=4).fit(MD.x)

table = np.column_stack((kmeans.labels_, gmm.predict(MD.x)))

In [ ]:
log_likelihood = MD_m4a.log_likelihood
print(log_likelihood)

In [ ]:
like_counts = mcdonalds['Like'].value_counts().sort_index(ascending=False)
like_counts.index.name = 'Like.n'
like_counts.reset_index(inplace=True)
like_counts['Like.n'] = 6 - like_counts['Like.n']
like_counts.set_index('Like.n', inplace=True)
print(like_counts)

In [ ]:
f = "+".join(list(mcdonalds.columns)[:11])
f = "Like.n ~ " + f
f = pd.Formula(f)
f


In [ ]:
np.random.seed(1234)
MD_reg2 = GaussianMixture(n_components=2, n_init=10, verbose=0).fit(mcdonalds[f])
print(MD_reg2)

In [ ]:
np.random.seed(1234)

# Assuming 'f' is the formula for regression model
# Assuming 'mcdonalds' is the DataFrame containing the data

# Convert the DataFrame to a numpy array
data = mcdonalds.to_numpy()

# Extract the response variable from the data
response = data[:, 0]

# Extract the predictor variables from the data
predictors = data[:, 1:]

# Fit the Gaussian Mixture Model
gmm = GaussianMixture(n_components=2, n_init=10, random_state=1234)
gmm.fit(predictors)

# Get the cluster labels
clusters = gmm.predict(predictors)

# Add the cluster labels to the DataFrame
mcdonalds['Cluster'] = clusters

print(mcdonalds)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor

np.random.seed(1234)

# Assuming 'f' is the formula for regression model
# Assuming 'mcdonalds' is the DataFrame containing the data

# Convert the DataFrame to a numpy array
data = mcdonalds.to_numpy()

# Extract the response variable from the data
response = data[:, 0]

# Extract the predictor variables from the data
predictors = data[:, 1:]

# Fit the Gaussian Mixture Model
gmm = GaussianMixture(n_components=2, n_init=10, random_state=1234)
gmm.fit(predictors)

# Get the cluster labels
clusters = gmm.predict(predictors)

# Add the cluster labels to the DataFrame
mcdonalds['Cluster'] = clusters

# Fit the regression model with clustered data
regressor = LinearRegression()
clustered_reg = TransformedTargetRegressor(regressor=regressor,
                                           transformer=gmm)
clustered_reg.fit(predictors, response)

print(clustered_reg)

In [ ]:
import statsmodels.api as sm

MD_ref2 = sm.regression.linear_model.RegressionResults(refit(MD_reg2))
print(MD_ref2.summary())

In [ ]:
plt.plot(MD.ref2)
plt.show()

In [5]:
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
MD_vclust = linkage(np.transpose(MD_x), method='ward')
plt.bar(range(len(MD_k4)), MD_k4, color='gray', align='center')
plt.xticks(range(len(MD_k4)), MD_k4)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
MD_proj = MD_pca.transform(MD_x)

# Create a scatter plot
plt.scatter(MD_proj[:, 0], MD_proj[:, 1], c=MD_k4.labels_)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# Plot the projection axes
proj_axes = MD_pca.components_[:2]
for axis in proj_axes:
    plt.arrow(0, 0, axis[0], axis[1], color='r', width=0.1)

plt.show()

In [ ]:
import seaborn as sns

k4 = MD.k4.labels_
df = pd.DataFrame({'k4': k4, 'Like': mcdonalds['Like']})
sns.mosaicplot(data=df, index=['k4'], columns=['Like'], fill=True, title="", xlabel="segment number")


In [ ]:

pd.crosstab(k4, mcdonalds['Gender']).apply(lambda r: r/r.sum(), axis=1).plot(kind='bar', stacked=True)
plt.show()


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [ ]:
X = mcdonalds[['Like.n', 'Age', 'VisitFrequency', 'Gender']]
y = mcdonalds['k4']

clf = DecisionTreeClassifier()
clf = clf.fit(X, y)

fig, ax = plt.subplots(figsize=(12, 12))
tree.plot_tree(clf, ax=ax)
plt.show()

In [ ]:
visit = np.array(mcdonalds['VisitFrequency']).astype(float)
visit = np.mean(visit, axis=0)


In [ ]:
like = pd.DataFrame(mcdonalds.groupby(k4)['Like.n'].mean())
like


In [ ]:
female = mcdonalds['Gender'].eq('Female').groupby(mcdonalds['k4']).mean()


In [ ]:
visit = [2.5, 3.2, 4.1, 3.7]
like = [0.5, -1.2, 2.1, -2.8]
female = [0, 1, 1, 0]

plt.scatter(visit, like, s=10*np.array(female), alpha=0.5)
plt.xlim(2, 4.5)
plt.ylim(-3, 3)

for i, txt in enumerate(range(1, 5)):
    plt.annotate(txt, (visit[i], like[i]))

plt.show()